# Import Packages

In [ ]:
import numpy as np
import pandas as pd
from scipy.io import loadmat
import tensorflow.keras as keras
from sklearn.preprocessing import MinMaxScaler

# Dataset Path Setting
make sure the paths are correct.


In [ ]:
class Datasets:
    MFPT_BASE = "/media/sda1/dataset/MFPT/MFPT Fault Data Sets"
    MFPT_DATA = "1 - Three Baseline Conditions"
    MFPT_MAT = "baseline_1.mat"
    MFPT_ABNORMAL_DATA = "2 - Three Outer Race Fault Conditions"
    MFPT_ABNORMAL_MAT = "OuterRaceFault_1.mat"

# Data Preprocessing
Preprocessing datas before the training
Simply change dataset shape in this example.

In [ ]:
class Preprocessing:

    @staticmethod
    def create_dataset(data, window_size, preprocess=False):
        org_shape = data.shape
        truncated = data.size - (int(data.size/window_size) * window_size)
        data = data[:data.size - truncated]
        
        if preprocess:
            minmax = MinMaxScaler()
            data = minmax.fit_transform(data)
        dataset = np.empty((int(data.size/window_size), window_size, 1))
        j = 0
        for i in [n*window_size for n in range(int(data.size/window_size))]:
            dataset[j] = data[i:i+window_size]
            j = j + 1
            
        print(f"Dataset shape changed {org_shape} -> {dataset.shape}.")
        print(f"Truncated {truncated} values ")
        #print(f"{data[data.size-100:data.size-50]}, {dataset[-2]}")
        #print(f"{data[data.size-50:data.size]}, {dataset[-1]}")
        return dataset
    
    @staticmethod
    def create_overlap_dataset(data, window_size, preprocess=False):
        org_shape = data.shape
        if preprocess:
            minmax = MinMaxScaler()
            data = minmax.fit_transform(data)
        dataset = np.empty((data.size-window_size+1, window_size, 1))
        j = 0
        for i in range(data.size-window_size+1):
            dataset[j] = data[i:i+window_size]
            j = j + 1
        print(f"Dataset shape changed {org_shape} -> {dataset.shape}")
        #print(f"{data[data.size-100:data.size-50]}, {dataset[-2]}")
        #print(f"{data[data.size-50:data.size]}, {dataset[-1]}")
        return dataset

# Load Dataset

### Load Anomaly Dataset

In [ ]:
import os
path = os.path.join(Datasets.MFPT_BASE, Datasets.MFPT_ABNORMAL_DATA, Datasets.MFPT_ABNORMAL_MAT)

mat = loadmat(path)
keys = mat['bearing'].dtype.fields
mat_dict = {
    list(keys.keys())[0] : mat['bearing'][0][0][0],
    list(keys.keys())[1] : mat['bearing'][0][0][1],
    list(keys.keys())[2] : mat['bearing'][0][0][2],
    list(keys.keys())[3] : mat['bearing'][0][0][3]
}
anomaly_dataset = Preprocessing.create_dataset(mat_dict['gs'], 50, True)

### Load Normal Dataset

In [ ]:
import os
path = os.path.join(Datasets.MFPT_BASE, Datasets.MFPT_DATA, Datasets.MFPT_MAT)

mat = loadmat(path)
keys = mat['bearing'].dtype.fields
mat_dict = {
    list(keys.keys())[0] : mat['bearing'][0][0][0],
    list(keys.keys())[1] : mat['bearing'][0][0][1],
    list(keys.keys())[2] : mat['bearing'][0][0][2],
    list(keys.keys())[3] : mat['bearing'][0][0][3]
}
normal_dataset = Preprocessing.create_dataset(mat_dict['gs'], 50)

# Load Model & Weights

### Load Model

In [ ]:
DIR = "./results"
MODEL_NAME = "MFPT_lstmAE_20_50_2022_02_03_20:11"
model = keras.models.load_model(f'{DIR}/{MODEL_NAME}')
model.summary()

### Calculate Loss for Anomaly Data

In [ ]:
anomaly_loss_list = []
i = 0
print("evaluating...")
for data in anomaly_dataset:
    data = np.reshape(data, (1,anomaly_dataset.shape[1], anomaly_dataset.shape[2]))
    loss = model.evaluate(data, data, verbose=0)
    anomaly_loss_list.append(loss)
    print(f"{i} / {anomaly_dataset.shape[0]} | {int(i / anomaly_dataset.shape[0] * 100)}%", end = '\r')
    i = i+1

### Calculate Loss For Normal Data

In [ ]:
normal_loss_list = []
i = 0
print("evaluating...")
for data in normal_dataset:
    data = np.reshape(data, (1,normal_dataset.shape[1], normal_dataset.shape[2]))
    loss = model.evaluate(data, data, verbose=0)
    normal_loss_list.append(loss)
    print(f"{i} / {normal_dataset.shape[0]} | {int(i / normal_dataset.shape[0] * 100)}%", end = '\r')
    i = i+1

### Save Result

In [ ]:
normal_loss_list = np.array(normal_loss_list)
np.save(f'{DIR}/{MODEL_NAME}/{MODEL_NAME}_{Datasets.MFPT_MAT}_Normal_losses.npy', normal_loss_list)

anomaly_loss_list = np.array(anomaly_loss_list)
np.save(f'{DIR}/{MODEL_NAME}/{MODEL_NAME}_{Datasets.MFPT_ABNORMAL_MAT}_losses.npy', anomaly_loss_list)

# Load Results

In [ ]:
DIR = "./results"
MODEL_NAME = "MFPT_lstmAE_10_50_2022_01_25_15:54"
NORMAL_FILE_NAME = "MFPT_lstmAE_10_50_2022_01_25_15:54_baseline_1.mat_Normal_losses.npy"
ANOMALY_FILE_NAME = "MFPT_lstmAE_10_50_2022_01_25_15:54_OuterRaceFault_1.mat_losses.npy"

In [ ]:
DIR = "./results"
MODEL_NAME = "MFPT_lstmAE_5_50_2022_01_25_15:34"
NORMAL_FILE_NAME = "MFPT_lstmAE_5_50_2022_01_25_15:34_baseline_1.mat_Normal_losses.npy"
ANOMALY_FILE_NAME = "MFPT_lstmAE_5_50_2022_01_25_15:34_OuterRaceFault_1.mat_losses.npy"

In [ ]:
normal_loss_list = np.load(f'{DIR}/{MODEL_NAME}/{NORMAL_FILE_NAME}')
anomaly_loss_list = np.load(f'{DIR}/{MODEL_NAME}/{ANOMALY_FILE_NAME}')

# Set Threshold

In [ ]:
threshold = np.mean(normal_loss_list) + 2*np.std(normal_loss_list)
threshold_max = np.array(normal_loss_list).max()

# Visualization

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(figsize = (20,10)) # This is your answer to resize the figure
# The below will help you expand on your question and resize individual elements within your figure. Experiement with the below parameters.
axs.set_title(MODEL_NAME, fontsize = 17.5)
axs.tick_params(axis = 'x', labelsize = 14)
axs.set_xlabel('Timestamp', size = 15)
axs.tick_params(axis = 'y', labelsize =14)
axs.set_ylabel('Loss', size = 15)
plt.plot(anomaly_loss_list, 'r', linewidth=1, label="Anomaly")
plt.plot(normal_loss_list, 'g', linewidth=1, label="Normal")
plt.hlines(threshold,0,np.array(normal_loss_list).size,'y', label="Threshold")
plt.hlines(threshold_max,0,np.array(normal_loss_list).size,'y', label="Normal Max")
plt.legend()
plt.savefig(f"{DIR}/{MODEL_NAME}/{MODEL_NAME}_Anomaly_Loss_Graph.png")